In [ ]:
pip install beautifulsoup4

In [ ]:
pip install requests

In [113]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import time
import csv

# def products_links(url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')

#     # print(soup.prettify())

#     # Targeting the <ul> element with id "product-grid"
#     product_grid = soup.find('ul', id='product-grid')

#     # Targeting the <li> elements within the <ul> element
#     li_elements = product_grid.find_all('li')

#     # Extracting the links from the <li> elements and joining with the base URL
#     base_url = 'https://watchcollectors.co.uk'  # Replace with the actual base URL
#     all_product = [urljoin(base_url, li.find('a')['href']) for li in li_elements if li.find('a')]

#     return all_product

def products_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Targeting the <ul> element with id "product-grid"
    product_grid = soup.find('ul', id='product-grid')

    # Targeting the <li> elements within the <ul> element
    li_elements = product_grid.find_all('li')

    # Extracting the links from the <li> elements and joining with the base URL
    base_url = 'https://watchcollectors.co.uk'  # Replace with the actual base URL
    all_product = [urljoin(base_url, li.find('a')['href']) for li in li_elements if li.find('a')]

    return all_product

def extract_all_product_links(base_url):
    all_product_links = []

    # Loop through pages (assuming there are 10 pages, you can modify the range accordingly)
    for page_number in range(1, 35):
        # Construct the URL for the current page
        page_url = f'{base_url}?page={page_number}'
        
        # Call the products_links function for the current page
        product_links_on_page = products_links(page_url)
        
        # Append the product links to the overall list
        all_product_links.extend(product_links_on_page)

    return all_product_links

# Replace 'https://watchcollectors.co.uk/collections/all' with the actual base URL
base_url = 'https://watchcollectors.co.uk/collections/all'
all_product_links = extract_all_product_links(base_url)

# Print the result
print(all_product_links)


['https://watchcollectors.co.uk/products/choaprd', 'https://watchcollectors.co.uk/products/louis-vuitton-ref-tambour-xl-minute-repeater-travel-time-18k-rose-gold-box-papers', 'https://watchcollectors.co.uk/products/rolex-lady-datejust-pearlmaster-ref-69298-white-mother-of-pearl-diamond-dial-18k-yellow-gold', 'https://watchcollectors.co.uk/products/rolex-yacht-master-40-ref-126622-slate-dial-box-papers-2021-stainless-steel-platinum', 'https://watchcollectors.co.uk/products/audemars-piguet-royal-oak-chronograph-ref-26603st-oo-d002cr-01000-blue-dial-box-papers-2010-stainless-steel', 'https://watchcollectors.co.uk/products/rolex-daytona-ref-116520-black-dial-2012-box-papers-stainless-steel-1', 'https://watchcollectors.co.uk/products/rolex-daytona-beach-blue-ref-116509-18k-white-gold-2007', 'https://watchcollectors.co.uk/products/vacheron-constantin-malte-chronograph-ref-49180-champagne-dial-40mm-18k-rose-gold', 'https://watchcollectors.co.uk/products/cartier-tank-americaine-chronograph-ref

In [114]:
len(all_product_links)

408

In [115]:
import warnings

# Suppress FutureWarnings for :contains
warnings.filterwarnings("ignore", category=FutureWarning, module="soupsieve")

imgUrls=[];productName=[];Price=[];modelNumber=[];modelYear=[];Gender=[];Diameter=[]
originalBox=[];originalPaper=[];productUrls=[]

def Product_details(links):
    c = 0
    for i in links:

        response = requests.get(i)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        try:
            image_urls = [img['data-src'] for img in soup.select('.media img[data-src]')]
            imgUrls.append(image_urls)
        except:
            imgUrls.append('')

        productUrls.append(i)
        
        try:
            product_title = soup.select_one('.product__title').get_text(strip=True)
            productName.append(product_title)

        except:
            productName.append('')
            
        try:
            price = soup.find('span', {'class': 'price-item--regular'}).text.strip()
            print(price)
            Price.append(price)
        except:
            Price.append('')

        
        try:
            model_number = soup.find('td', string='Model Number').find_next('td').get_text(strip=True)
            modelNumber.append(model_number)
        except:
            modelNumber.append('')
    
        try:
            model_year = soup.find('td', string='Year').find_next('td').get_text(strip=True)
            modelYear.append(model_year )
        except:
            modelYear.append('')
    
        try:
            gender = soup.find('td', string='Gender').find_next('td').get_text(strip=True)
            Gender.append(gender)
        except:
            Gender.append('')

        try:
            diameter = soup.find('td', string='Diameter (mm)').find_next('td').get_text(strip=True)
            Diameter.append(diameter)
        except:
            Diameter.append('')

        try:
            original_box = soup.find('td', string='Original Box').find_next('td').get_text(strip=True)
            originalBox.append(original_box)
        except:
            originalBox.append('')
        
        try:
            original_papers = soup.find('td', string='Original Papers').find_next('td').get_text(strip=True)
            originalPaper.append(original_papers)
        except:
            originalPaper.append('')
        
        c = c + 1
        print(f"Product {c} completed.")

        time.sleep(5)  # Sleep for 5 seconds between requests
        
        if c==15:
            break
        
    # Writing to CSV file
    with open('output.csv', 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Image Urls', 'Product Name', 'Price', 'Model Number', 'Model Year', 'Gender', 'Diameter',
                        'Original Box', 'Original Papers', 'Product Url'])

        for i in range(len(productName)):
            # Join image URLs into a single string separated by commas
            img_urls_str = ', '.join(imgUrls[i])
            writer.writerow([img_urls_str, productName[i], Price[i], modelNumber[i], modelYear[i], Gender[i],
                            Diameter[i], originalBox[i], originalPaper[i], productUrls[i]])
    return True

In [105]:
# url = input("Enter Url: ")
# all_links = products_links(url)

# # Print the list of links
# print(all_links)


['https://watchcollectors.co.uk/products/choaprd', 'https://watchcollectors.co.uk/products/louis-vuitton-ref-tambour-xl-minute-repeater-travel-time-18k-rose-gold-box-papers', 'https://watchcollectors.co.uk/products/rolex-lady-datejust-pearlmaster-ref-69298-white-mother-of-pearl-diamond-dial-18k-yellow-gold', 'https://watchcollectors.co.uk/products/rolex-yacht-master-40-ref-126622-slate-dial-box-papers-2021-stainless-steel-platinum', 'https://watchcollectors.co.uk/products/audemars-piguet-royal-oak-chronograph-ref-26603st-oo-d002cr-01000-blue-dial-box-papers-2010-stainless-steel', 'https://watchcollectors.co.uk/products/rolex-daytona-ref-116520-black-dial-2012-box-papers-stainless-steel-1', 'https://watchcollectors.co.uk/products/rolex-daytona-beach-blue-ref-116509-18k-white-gold-2007', 'https://watchcollectors.co.uk/products/vacheron-constantin-malte-chronograph-ref-49180-champagne-dial-40mm-18k-rose-gold', 'https://watchcollectors.co.uk/products/cartier-tank-americaine-chronograph-ref

In [116]:
# Product_details(all_links)
Product_details(all_product_links)

£165,000.00
Product 1 completed.
£165,000.00
Product 2 completed.
£165,000.00
Product 3 completed.
£165,000.00
Product 4 completed.
£165,000.00
Product 5 completed.
£165,000.00
Product 6 completed.
£165,000.00
Product 7 completed.
£165,000.00
Product 8 completed.
£165,000.00
Product 9 completed.
£165,000.00
Product 10 completed.
£165,000.00
Product 11 completed.
£165,000.00
Product 12 completed.
£165,000.00
Product 13 completed.
£165,000.00
Product 14 completed.
£165,000.00
Product 15 completed.


True